In [2]:
import tweepy
import pandas as pd
import sqlite3
import datetime

# Creating a SQL connection to our SQLite database
con = sqlite3.connect("twitter.sqlite")

#store key screts for authentication
consumer_key = "1fjQSB7TQLPhhhGPz2PjQMEMz"
consumer_secret = "fHO72DEe4vuw0m1fzJ7f89JpERSgUg4OA5sNKRCi0TO0IElXR0"
access_token = "1487994957327749123-eBuh4zylO1WvMpGi8OWDIOclpMndJB"
access_token_secret = "QbhohHu2bOfvLA9ElF4FSBS15vUiF8sRwbDFOZjo0UOn1"

#initialize authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)


#designate last tweet to append new tweets to database
df = pd.read_sql('select * from twitter_table', con)
last_id = df.Since_Id.max()

#fetch mainstream tweets
tweets = []
likes = []
time = []
name = []
dateid = []
listm = {
    'ABC', 'CBSNews', 'CNN', 'FoxNews', 'MSNBC', 'NBCNews', 'nytimes',
    'USATODAY', 'WSJ', 'washingtonpost', 'politico', 'BGOV', 'nypost', 'Newsy',
    'CNBC', 'nprpolitics', 'NewsHour', 'NewYorker', 'WIRED', 'voxdotcom',
    'chicagotribune', 'UnivisionNews', 'BuzzFeedNews', 'Newsweek', 'usnews',
    'thehill', 'AP', 'TheAtlantic', 'thedailybeast', 'EpochTimes', 'newsmax',
    'latimes', 'BostonGlobe', 'TIME', 'dcexaminer', 'RollingStone', 'Slate',
    'realDailyWire', 'dallasnews', 'VanityFair', 'townhallcom', 'TechCrunch',
    'HeavySan', 'intelligencer', 'Inc', 'BlazeTV', 'StarTribune',
    'ScienceMagazine', 'MashableNews', 'FortuneMagazine', 'Forbes',
    'barronsonline', 'seattletimes', 'sciam', 'axios', 'SmithsonianMag',
    'FiveThirtyEight', 'OANN', 'PhillyInquirer', 'TheCut', 'FastCompany', 'qz',
    'MiamiHerald', 'statnews', 'sfchronicle', 'DailyCaller', 'WashTimes',
    'theskimm', 'NRO', 'Deseret', 'FDRLST', 'TexasTribune', 'TheWeek',
    'baltimoresun', 'TB_Times', 'TeenVogue', 'cspan', 'BreitbartNews',
    'Newsday', 'TheAdvocateMag', 'theintercept', 'sltrib', 'TheRoot',
    'Suntimes', 'propublica', 'HoustonChron', 'sdut', 'AmericanThinker',
    'PopSci', 'reason', 'MotherJones', 'newsobserver', 'ForeignPolicy',
    'MorningBrew', 'VICENews', 'readersdigest', 'reviewjournal', 'techreview',
    'PunchbowlNews', 'thenation', 'bangordailynews', 'NorthBayNews',
    'newrepublic', 'FreeBeacon', 'amconmag', 'ArkansasOnline', 'ScienceNews',
    'PressHerald', 'csmonitor', 'cnsnews', 'CityJournal', 'Poynter',
    'protocol', 'Harpers', 'TheSpectator', 'EconUS'
}
for key in listm:
    j = tweepy.Cursor(
        api.search_tweets,
        q="<from:" + key + "> <covid OR sars-cov-2 OR chinavirus -RT> ",
        since_id=last_id,
        tweet_mode="extended")
    for i in j.items():
        tweets.append(i.full_text)
        likes.append(i.favorite_count)
        time.append(i.created_at)
        name.append(i.user.screen_name)
        dateid.append(i.id)

#write MS tweets to data frame
df = pd.DataFrame({
    "Tweets": tweets,
    "Likes": likes,
    "Time": time,
    "Name": name,
    "Since_Id": dateid
})
df['Source Type'] = pd.Series(["Mainstream" for x in range(len(df.index))])

# Appending pandas dataframe to sql database
df.to_sql('twitter_table', con, index=False, if_exists='append')

#fetch politician tweets
tweets = []
likes = []
time = []
name = []
dateid = []
listp = {
    'SenTedCruz', 'SenatorHagerty', 'JohnCornyn', 'AlexPadilla4CA',
    'marcorubio', 'MarshaBlackburn', 'SenatorDurbin', 'SenatorShaheen',
    'SenJeffMerkley', 'senrobportman', 'RepCasten', 'RepTedLieu',
    'RepDonBacon', 'RepJennif', 'RepBoebert', 'RepChipRoy', 'RepTimBurchett',
    'RepJayapal', 'RepMTG', 'RepMattGaetz', 'AOC', 'RepCawthorn'
}
for key in listp:
    j = tweepy.Cursor(
        api.search_tweets,
        q="<from:" + key + "> <covid OR sars-cov-2 OR chinavirus -RT> ",
        since_id=last_id,
        tweet_mode="extended")
    for i in j.items():
        tweets.append(i.full_text)
        likes.append(i.favorite_count)
        time.append(i.created_at)
        name.append(i.user.screen_name)
        dateid.append(i.id)

#write POLIT tweets to data frame
df = pd.DataFrame({
    "Tweets": tweets,
    "Likes": likes,
    "Time": time,
    "Name": name,
    "Since_Id": dateid
})
df['Source Type'] = pd.Series(["Politicians" for x in range(len(df.index))])

# Appending pandas dataframe to sql database
df.to_sql('twitter_table', con, index=False, if_exists='append')

#fetch scientific tweets
tweets = []
likes = []
time = []
name = []
dateid = []

st = tweepy.Cursor(
    api.search_tweets,
    q="<from:CDCgov OR from:NIH> <covid OR sars-cov-2 OR chinavirus -RT> ",
    since_id=last_id,
    tweet_mode="extended")

for i in st.items():
    tweets.append(i.full_text)
    likes.append(i.favorite_count)
    time.append(i.created_at)
    name.append(i.user.screen_name)
    dateid.append(i.id)

#write SCI tweets to data frame
df = pd.DataFrame({
    "Tweets": tweets,
    "Likes": likes,
    "Time": time,
    "Name": name,
    "Since_Id": dateid
})
df['Source Type'] = pd.Series(["Scientific" for x in range(len(df.index))])

# Appending pandas dataframe to sql database
df.to_sql('twitter_table', con, index=False, if_exists='append')

#fetch fringe tweets
tweets = []
likes = []
time = []
name = []
dateid = []
listm = {
    'joerogan', 'TPUSA', 'Timcast', 'MsBlaireWhite', 'TheOfficerTatum',
    'KeithOlbermann', 'hodgetwins', 'DrunkenPeasants', 'InnuendoStudios',
    'hasanthehun', 'TheOmniLiberal', 'ContraPoints', 'MarkDice',
    'KyleKulinski', 'TheLaurenChen', 'MattWalshBlog', 'andrewklavan',
    'Lauren_Southern', 'PhillyD', 'jordanbpeterson', 'dpakman', 'SamHarrisOrg',
    'jimmy_dore', 'majorityfm', 'rustyrockets', 'RudyGiuliani', 'AnnCoulter',
    'TheQuartering', 'SaltyCracker9', 'LiberalHivemind', 'DineshDSouza',
    'ANTHONYBLOGAN', 'VaushV', 'LeeCamp', 'amtvmedia', 'AydinPaladin',
    'AwakenWithJP', 'DiamondandSilk', 'TheYoungTurks', 'democracynow',
    'RealAmVoice', 'Suspended', 'BlacklistedNews', 'news_ntd', 'TonyBrunoShow',
    'WayneDupreeShow', 'JoeTalkShow', 'monicaonairtalk', 'ResisttheMS',
    'Breaking911', 'RSBNetwork', 'JustTheNews', 'TheDamaniFelder', 'hotair',
    'LifeNewsHQ', 'RebelNewsOnline', 'MalcolmOutLoud', 'SiriusXMPatriot',
    'MediaRightNews', 'Leeroypress', 'twitchy', 'DschlopeisBack',
    'VillageNorth', 'richardursomd', 'ajeemontes', 'CRRJA5', 'zerohedge',
    'tspooky', 'ROHLL5', 'carlossimancas', 'ivoryhecker', 'William5849',
    'Dloesch', 'LadyRedWave', 'Gonewiththerain', 'RealStewPeters',
    'bigleaguepol', 'SteveGruber', 'tweettruth2me', 'chiefnerd',
    'DavidJHarrisJr', 'mythinformedmke', 'brandonstraka', 'joeysalads',
    'realmattcouch', 'serremmy', 'redvoicemedia', 'mikhailaaleksis',
    'jasonmillerindc', 'childrensHD', 'barbiewakeupusa'
}
for key in listm:
    j = tweepy.Cursor(
        api.search_tweets,
        q="<from:" + key + "> <covid OR sars-cov-2 OR chinavirus -RT> ",
        since_id=last_id,
        tweet_mode="extended")
    for i in j.items():
        tweets.append(i.full_text)
        likes.append(i.favorite_count)
        time.append(i.created_at)
        name.append(i.user.screen_name)
        dateid.append(i.id)

#write fringe tweets to dataframe
df = pd.DataFrame({
    "Tweets": tweets,
    "Likes": likes,
    "Time": time,
    "Name": name,
    "Since_Id": dateid
})
df['Source Type'] = pd.Series(["Fringe" for x in range(len(df.index))])

# Appending pandas dataframe to sql database
df.to_sql('twitter_table', con, index=False, if_exists='append')

Rate limit reached. Sleeping for: 875


42